# Compare Flight Prices

The script here is a small exerpt that is used to automate the flight search flow and scrape the price and compare with that of the competitors. This involves collecting prices and flight info from competitor websites and compare it to that of Flight Centre. 

In this script below, I have removed any scraping code on competitor websites because they have since added a dissallow on their robot.txt. Even when allowed, precautions using delays were used to reduce the load on their servers. For this exercise, I've included a simple csv file with potential data from the other search engines.

You will note here that it is important to only compare the prices of the exact same flights. There are some cheaper suppliers that do not exist across these search engines, and while a separate case was made for that, the point of this exercise is to highlight potential differences in prices for the same flight. This is a part of complete 'scraping and compare' project. There will be a separate project added soon to show a more scalable and maintainable approach using constructor classes

> The original project was successful in discovering varous domestic and transborder examples where price discrepancies existed. The expectation for this fast solution was to repeat searches manually and then identify differences in base prices and taxes

### Import

We start by importing the libraries that will be used for analysis, automated website navigation, and web scraping. We also set the browser driver, which in this case is Chrome. 

> The driver for different browsers are available. We utilized the basic search function with different browsers for a regression test suite as part of a major release testing the new flight search engine

In [1]:
# Basic Imports
import pandas as pd
import numpy as np
import time
import datetime
# Load selenium components and browser driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
# For scraping
from bs4 import BeautifulSoup as soup

In [2]:
# Download and use specific browser driver - in this case 'Chrome' 
# NOTE: Switch to forward slash in path
chrome_driver_path = 'C:/Users/Raja/driver/chromedriver'

### Import scraped price data
This is the scraped price data from competitor sites. There is a considerable effort required for data wrangling and cleaning that is not covered in this exercise, but instead we will use the manulally produced data wto set the search criteria for searches in Flight Centre

The same methods used below to search and scrape on the FC.ca site was used on other sites. The use case used there will be used here, for the sake of simplicity: 
* Conduct a Domestic or Transborder search
* Select 'non-stop only'
* Select same carrier or mixed supplier for the major canadian and american suppliers eg: AC, WS, TS, DL, AA

In [3]:
# Competition data
df_comp = pd.read_csv('./data/Mock flights scrape.csv')

In [4]:
df_comp

,search_engine,departure_airport,return_airport,departure_date,arrival_date,flight_out,flight_in,price
0,Site1,YYZ,YVR,1/12/2021,1/19/2021,AC185,AC118,$571.43
1,Site2,YYZ,YVR,1/11/2021,1/19/2021,AC127,AC126,$335.02
2,Site2,YYZ,YVR,1/12/2021,1/19/2021,AC127,WS724,$481.50
3,Site1,YYZ,YVR,1/12/2021,1/19/2021,WS725,WS724,$484.23
4,Site2,YYZ,LAX,3/31/2021,4/14/2021,AC791,AC792,$328.33
5,Site1,YYZ,LAX,3/31/2021,4/14/2021,AC791,AC788,$328.33


Changing the datatype of date fields can be done during import of csv files. However, I've chosen to do this instead as this is applicable to any type of data

In [5]:
# Change to date type
df_comp['departure_date'] =  pd.to_datetime(df_comp['departure_date'], infer_datetime_format=True)
df_comp['arrival_date'] =  pd.to_datetime(df_comp['arrival_date'], infer_datetime_format=True)

In [6]:
# RESULT: Date type changed
df_comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 8 columns):
search_engine        6 non-null object
departure_airport    6 non-null object
return_airport       6 non-null object
departure_date       6 non-null datetime64[ns]
arrival_date         6 non-null datetime64[ns]
flight_out           6 non-null object
flight_in            6 non-null object
price                6 non-null object
dtypes: datetime64[ns](2), object(6)
memory usage: 464.0+ bytes


### Reusable Functions
It is very valuable to set up a framework for the different functions used repeatedly in different projects. The functionality that we are repeating for now is 'website navigation' and 'scraping'. The function below is only really used twice and it is not even relevant to the main purpose of this scope. But it provides relevant information for scalability and debugging as well as some general info if we were to sit through the process

> It is important to note that a lot of scripting below can be made more scalable and easier to maintain by using constructors. This is an ongoing process and I will include a second part to this in order to show the natural iteration process of taking a working script expected in a short time frame; eventually transforming into a more maintainable version with the success of the use case

> The function below was useful for debugging when flights that existed on competitor sites was not available on either the first page of FC or not available at all. This is not going to be difficult to re-create with limited competitor data, but it identifies a scenario where knowing what the crawler sees is very useful for debugging

In [22]:
def extract_flights_on_page(dataindex, segment):
    ''' Scrape all flight results on page identified with same class 'tripDetails'
        Function attribute 'dataindex' will uniquely identify input data used for search
        Function attribute 'segement' will differentiate outgoing and incoming flight search segments
        Data is output as a list 
    '''
    # Extract all trip details from flights results page
    results_page_soup = soup(driver.page_source, 'xml')
    trip_details = results_page_soup.select('div[class*="tripDetails"]')
    airline = []
    flight_number = []
    origin = []
    destination = []
    for trip in trip_details:
        airline.append(trip['data-airline'])
        flight_number.append(trip['data-flightnumber'])
        origin.append(trip['data-origin'])
        destination.append(trip['data-destination'])
    #Extract price from different element
    #Also add an identifier for which input data was used
    trip_price = results_page_soup.select('span[class*="test-priceWholeValue"]')
    pricelist = []
    test_stage = []
    for price in trip_price:
        pricelist.append(price.text.replace(",", ""))
        test_stage.append('dataindex' + str(dataindex) + '-' + segment )
    data = [airline,flight_number,origin,destination,pricelist,test_stage]
    return data

### Perform Search and Capture FC Price
The complete automated flight flow is run inside a loop in order to go through all the competition data. Here we will use the  flight details captured from the competitors websites to search and automate the flow to select the same flights and ultimately compare if the final prices are the same

> The expectation is that all search engines should provide the same prices for the same flights

Below is the first step at creating a scalable test script to run through the flight flow. Here we identify all the elements that are going to interacted with during the search flow. Keeping them together ensures that any FE changes are easy to keep up with

In [8]:
# FC search form locators
origin_field = (By.ID, 'js-fc-input-origin')
destination_field = (By.ID, 'js-fc-input-destination')
departure_date_field = (By.ID, 'js-fc-input-departure-date')
return_date_field = (By.ID, 'js-fc-input-return-date')
search_flight_button = (By.XPATH, '//button[contains(@class, "s-btn--submit")]')
departure_date_field = (By.ID, 'js-fc-input-departure-date')
# FC result page locators
nonstop_checkbox = (By.XPATH, '//span[text() = "Non-stop"]')

Output dataframe as a copy of the input dataframe with one column added to capture the FC equivalent price

In [9]:
# Data frame to be used to capture FC price
# In final line of test loop below, we will use 'df_out['fc_price'].iloc[[index]]'
df_out = df_comp.copy()
df_out['fc_price'] = 'NA'
df_out

,search_engine,departure_airport,return_airport,departure_date,arrival_date,flight_out,flight_in,price,fc_price
0,Site1,YYZ,YVR,2021-01-12,2021-01-19,AC185,AC118,$571.43,NA
1,Site2,YYZ,YVR,2021-01-11,2021-01-19,AC127,AC126,$335.02,NA
2,Site2,YYZ,YVR,2021-01-12,2021-01-19,AC127,WS724,$481.50,NA
3,Site1,YYZ,YVR,2021-01-12,2021-01-19,WS725,WS724,$484.23,NA
4,Site2,YYZ,LAX,2021-03-31,2021-04-14,AC791,AC792,$328.33,NA
5,Site1,YYZ,LAX,2021-03-31,2021-04-14,AC791,AC788,$328.33,NA


In [23]:
flight_options = [] # List to capture all flight options for debugging purposes
# Loop through all competitor data to perfrom SEARCH and capture equivalent FC price
for index in range(len(df_comp)):
    # Extract data to specify in search
    # airports specified from other website
    departure_airport = df_comp['departure_airport'][index]
    arrival_airport = df_comp['return_airport'][index]
    # Date specified from other website
    # Note: converting to 'int' removes leading zero | otherwise use .strip("0")
    dep_year = int((df_comp['departure_date'][index]).strftime('%Y'))
    dep_month = int((df_comp['departure_date'][index]).strftime('%m'))
    dep_day = int((df_comp['departure_date'][index]).strftime('%d'))
    arr_year = int((df_comp['arrival_date'][index]).strftime('%Y'))
    arr_month = int((df_comp['arrival_date'][index]).strftime('%m'))
    arr_day = int((df_comp['arrival_date'][index]).strftime('%d'))
    # Formulate date selection in unixtime
    dep_date = datetime.date(dep_year, dep_month, dep_day)
    dep_unixtime = int(time.mktime(dep_date.timetuple()))  
    arr_date = datetime.date(arr_year, arr_month, arr_day)
    arr_unixtime = int(time.mktime(arr_date.timetuple()))
    
    ################## STEP 1: Enter search fields based on competition data
    # Note: Included sleep between selecting departure field and selecting suggested airports
    # Note: odd workaround for explicit wait - cannot use pre-set locators AND have to use extra brackets
    # Note: Should be constructed in a function
    driver = webdriver.Chrome(chrome_driver_path)
    driver.get('https://www.flightcentre.ca')
    driver.maximize_window()
    driver.find_element(*origin_field).send_keys(departure_airport)
    time.sleep(2)
    driver.find_element(*origin_field).send_keys(Keys.RETURN)
    driver.find_element(*destination_field).send_keys(arrival_airport)
    time.sleep(2)
    driver.find_element(*destination_field).send_keys(Keys.ENTER)
    driver.find_element(*departure_date_field).click()
    time.sleep(1)
    # enter explicit date year
    driver.find_element_by_xpath('//button[@class = "dp-cal-year"][1]').click()
    driver.find_element_by_xpath('//button[@data-year = "' + str(dep_year) + '"]').click()
    # enter explicit date month
    driver.find_element_by_xpath('//button[@class = "dp-cal-month"][1]').click()
    driver.find_element_by_xpath('//button[@data-month = "' + str(dep_month - 1) + '"]').click()
    # enter explicit date - based on unix time | add three zeroes because thats how they are marked for some reason
    # Also need to make sure to include 'not' in xpath to disregard empty date selector in previous month
    driver.find_element_by_xpath('//button[@data-date = "' + str(dep_unixtime) + \
                                 '000" and not(contains(@class, "dp-edge-day"))]').click()
    # Repeat all for arrival date seletion
    driver.find_element_by_xpath('//button[@class = "dp-cal-year"][1]').click()
    driver.find_element_by_xpath('//button[@data-year = "' + str(arr_year) + '"]').click()
    driver.find_element_by_xpath('//button[@class = "dp-cal-month"][1]').click()
    driver.find_element_by_xpath('//button[@data-month = "' + str(arr_month - 1) + '"]').click()
    driver.find_element_by_xpath('//button[@data-date = "' + str(arr_unixtime) + \
                                 '000" and not(contains(@class, "dp-edge-day"))]').click()
    ################## STEP 2: Include enough gap between fields being entered and PERFORM SEARCH
    time.sleep(2)
    driver.find_element(*search_flight_button).click()
    # Result: Flights results open on new tab - So switch focus
    driver.switch_to.window(driver.window_handles[-1])
    ################## STEP 3: Filter non-stop outgoing flights only
    # Set an explicit wait for new tab to load - fallback if element is not found is 10
    # And once found - Check the box
    element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//span[text() = "Non-stop"]')))
    element.click()
    ################# ASIDE: Provide an implicit wait and capture all options on page for debugging
    # Flight out options (page 1 only)
    flight_options.append(extract_flights_on_page(index,'outgoing'))
    ################## STEP 4: Using input data select specific flight
    airline_out = df_comp['flight_out'][index][:2]
    number_out = df_comp['flight_out'][index][-3:]
    airline_in = df_comp['flight_in'][index][:2]
    number_in = df_comp['flight_in'][index][-3:]
    # select outgoing airline
    driver.find_element_by_xpath('//div[@data-airline = "' + airline_out + '" and @data-flightnumber = "' \
                                 + number_out + '"]').click()
    ################## STEP 5: Select cheapest class option tile   
    time.sleep(2)
    # Unique step - On FC on selecting a flight, there is an upsell tactic that provides all classes
    # Keeping in line with simplicity and most likely scenario, we select the cheapest class
    driver.find_element_by_xpath('//button[contains(@class, "test-fareTypeFooterButton")][1]').click()
    # Result: Wait for page with incoming flights to load                      
    element = WebDriverWait(driver, 20).until(EC.presence_of_element_located\
                                              ((By.XPATH, '//div[starts-with(@class, "fullJourneyInfo")]')))
    ################## STEP 6: Filter non-stop incoming flights only    
    # Note here that a different element was used
    driver.find_element_by_xpath('//*[contains(@class, "test-direct")]').click()
    ################## ASIDE: Capture all flight options on incoming segment using same function
    flight_options.append(extract_flights_on_page(index,'incoming'))                         
    ################## STEP 7: Set an implicit wait and select input data specified incoming flight
    time.sleep(2)
    driver.find_element_by_xpath('//div[@data-airline = "' + airline_in + '" and @data-flightnumber = "' \
                                 + number_in + '"]').click()
    ################## STEP 8: Repeat same step to select cheapest option
    time.sleep(2)
    driver.find_element_by_xpath('//button[contains(@class, "test-fareTypeFooterButton")][1]').click()
    ################## STEP 9: FINAL STEP - Wait for Review page load and capture the final price
    element_pageheader = WebDriverWait(driver, 20).until(EC.presence_of_element_located\
                                                         ((By.XPATH, '//*[starts-with(@class, "reviewHeader")]')))
    element_price = WebDriverWait(driver, 20).until(EC.element_to_be_clickable\
                                                    ((By.XPATH, '//span[contains(@class, "priceWholeValue")]')))
    df_out['fc_price'].iloc[[index]] = element_price.text
    # Close selenium driver
    driver.close()
    driver.quit()

### Results

In [24]:
df_out

,search_engine,departure_airport,return_airport,departure_date,arrival_date,flight_out,flight_in,price,fc_price
0,Site1,YYZ,YVR,2021-01-12,2021-01-19,AC185,AC118,$571.43,571
1,Site2,YYZ,YVR,2021-01-11,2021-01-19,AC127,AC126,$335.02,390
2,Site2,YYZ,YVR,2021-01-12,2021-01-19,AC127,WS724,$481.50,484
3,Site1,YYZ,YVR,2021-01-12,2021-01-19,WS725,WS724,$484.23,484
4,Site2,YYZ,LAX,2021-03-31,2021-04-14,AC791,AC792,$328.33,328
5,Site1,YYZ,LAX,2021-03-31,2021-04-14,AC791,AC788,$328.33,328


In [44]:
df_out.iloc[1]

search_engine                      Site2
departure_airport                    YYZ
return_airport                       YVR
departure_date       2021-01-11 00:00:00
arrival_date         2021-01-19 00:00:00
flight_out                         AC127
flight_in                          AC126
price                           $335.02 
fc_price                             390
Name: 1, dtype: object

> Further exploring the price difference between FC and competitor price, it was determined that taxes charged on the competitor site is lower due to flight segments being split into 'one-way' segments and charging lower 'British Columbia' province tax rates on the return segment, while FC charged 'Ontario' tax rates on treating the pricing as one

> This is the case for all the other examples, but an exception or 'tax loop hole' was utilized by the competitors